<font size="36">1. Import Libraries</font>

In [23]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import datetime

import pandas as pd

from tensorflow import keras 

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

from keras.preprocessing import image

from keras.preprocessing.image import ImageDataGenerator

import pickle

In [24]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils
mp_face_mesh=mp.solutions.face_mesh

In [25]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #Color Conversion BGR 2 RGB
    image.flags.writeable=False # Image is no longer writeable
    results=model.process(image)    # Make prediction
    image.flags.writeable=True  # Image is now writeable
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Color Conversion RGB w BGR
    return image,results

In [26]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )

In [27]:
def extract_keypoints(results):
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    
   
    return np.concatenate([face])

In [28]:
# Path for exported data, numpy arrays
DATA_PATH=os.path.join('Training_Data')

# Actions that we try to detect
actions=np.array(['active','bored','distracted'])

#Thirty video worth of data
no_sequences=30

# Videos are going to be 30 franes in length
sequence_length=30

In [29]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [30]:
# Load the models
face_prediction_model=keras.models.load_model('face_predictor.h5')
engagement_prediction_model=keras.models.load_model('engagement_predictor.h5')

In [31]:
# Create the database for the results
import sqlite3

connection = sqlite3.connect("engagement.db")

cur = connection.cursor()

cur.execute( "CREATE TABLE if not exists experiments (Experiment_id integer default 1,Experiment_time text, result_id integer primary key AUTOINCREMENT, StudentName text, EngagementType text)")
connection.commit()

In [32]:
result=cur.execute("select max(Experiment_id) from experiments")


experiment_id=cur.fetchone()[0]


#experiment_id+=1
if (experiment_id==None):
    experiment_id=1
    
else:
    experiment_id+=1

print(experiment_id)


3


In [33]:
sequence = []

# Create a folder to store the main folder of the faces
ct=datetime.datetime.now()
main_folder_path='Experiment '+ct.strftime("%m-%d-%Y-%H-%M-%S")
os.mkdir(main_folder_path)

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

cap = cv2.VideoCapture('Videos/Experiment 2.mp4')


# read from the pickle file
ResultMap=pd.read_pickle(r'ResultsMap.pkl')


# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    face_mesh=mp_face_mesh.FaceMesh(max_num_faces=1,static_image_mode=True,refine_landmarks=True,min_detection_confidence=0.5,min_tracking_confidence=0.5) 
    while cap.isOpened():
        
        # Read feed
        ret, frames = cap.read()
        
        # Close the video automatically
        if (ret is False):
            break
     
        gray=cv2.cvtColor(frames,cv2.COLOR_BGR2GRAY)
        
        # Detect the faces
        faces=faceCascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=2,minSize=(30,30))
        
        xcoords=[]
        ycoords=[]
        
        
        ct=datetime.datetime.now()
        faces_folder_path=main_folder_path+'/Faces'+ct.strftime("%m-%d-%Y-%H-%M-%S")
        os.mkdir(faces_folder_path)
        
        counter=0
        for (x,y,w,h) in faces:
            face = frames[y:y+h, x:x+w] #slice the face from the image
            # save each image in a separate file
            cv2.rectangle(frames, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.imwrite(faces_folder_path+'/face'+str(counter)+'.jpg',face)
            
            # Add the positions of the faces into a list
            xcoords.append(x)
            ycoords.append(y)
            
            counter=counter+1

           #########################################################################

        '''########### Making single predictions ###########'''
        for c in range(counter):
            ImagePath=faces_folder_path+'/face'+str(c)+'.jpg'
            img=cv2.imread(ImagePath)
            
            test_image=keras.preprocessing.image.load_img(ImagePath,target_size=(64, 64))
            test_image=keras.preprocessing.image.img_to_array(test_image)

            test_image=np.expand_dims(test_image,axis=0)

            face_prediction_result=face_prediction_model.predict(test_image,verbose=0)
            
            #########################################################################
            # Make detections
            image, results = mediapipe_detection(img, holistic)
            #print(results)

            # Draw landmarks
            draw_styled_landmarks(img, results)
            
            # Save to the file
            #cv2.imwrite(faces_folder_path+'/face'+str(c)+'.jpg',img)
            cv2.imwrite(faces_folder_path+'/face'+str(c)+'.jpg',img)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            
            #print(actions[np.argmax(res)])
          
            if len(sequence) == 30:
                face_prediction_result=face_prediction_model.predict(test_image,verbose=0)
                res = engagement_prediction_model.predict(np.expand_dims(sequence, axis=0))[0]
                print("Face:(" +str(c)+"):"+ResultMap[np.argmax(face_prediction_result)])                
                print(actions[np.argmax(res)])
                
                
                #cv2.putText(frames,ResultMap[np.argmax(face_prediction_result)]+ ' '+actions[np.argmax(res)], (xcoords[c],ycoords[c]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.putText(frames,ResultMap[np.argmax(face_prediction_result)]+ ' '+actions[np.argmax(res)], (xcoords[c],ycoords[c]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                                
                # Add the prediction results to the database
                #print("INSERT INTO experiments (Experiment_id,Experiment_time,StudentName,EngagementType ) values("+str(experiment_id)+",'"+ct.strftime("%m-%d-%Y-%H-%M-%S")+"','"+ResultMap[np.argmax(face_prediction_result)]+"','"+actions[np.argmax(res)]+"')")
                
                cur.execute("INSERT INTO experiments (Experiment_id,Experiment_time,StudentName,EngagementType ) values("+str(experiment_id)+",'"+ct.strftime("%m-%d-%Y-%H-%M-%S")+"','"+ResultMap[np.argmax(face_prediction_result)]+"','"+actions[np.argmax(res)]+"')")
                connection.commit()
                
                
            # Show to screen
            cv2.namedWindow("Main Window", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("Main Window", 900, 500)
            cv2.imshow('Main Window', frames)
            
            
        # Break gracefully
        if cv2.waitKey(1000) & 0xFF == ord('q'):
            break
    connection.close
    cap.release()
    cv2.destroyAllWindows()

C:\Users\sriha\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 [==============================] - 0s 325ms/step
Face:(0):Mohamed
active
1/1 [==============================] - 0s 25ms/step
Face:(1):Mohamed
active
1/1 [==============================] - 0s 26ms/step
Face:(2):Mohamed
active
1/1 [==============================] - 0s 27ms/step
Face:(0):Mohamed
active
1/1 [==============================] - 0s 26ms/step
Face:(1):Mohamed
active
1/1 [==============================] - 0s 25ms/step
Face:(0):Mohamed
active
1/1 [==============================] - 0s 24ms/step
Face:(1):Mohamed
distracted
1/1 [==============================] - 0s 29ms/step
Face:(2):Mohamed
active
1/1 [==============================] - 0s 21ms/step
Face:(0):Mohamed
active
1/1 [==============================] - 0s 26ms/step
Face:(1):Mohamed
active
1/1 [==============================] - 0s 22ms/step
Face:(2):Mohamed
active
1/1 [==============================] - 0s 21ms/step
Face:(0):Mohamed
active
1/1 [==============================] - 0s 22ms/step
Face:(1):Mohamed
active
1/1 [==